Code is heavily based on this example:
https://keras.io/examples/structured_data/structured_data_classification_from_scratch/


With some light modifications using a different dataset and using a regression algorithim instead of classification

This is really just a way for me to test webhosting a model on github.io

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers

2022-01-08 00:21:12.388791: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-08 00:21:12.388821: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
file_url = "/home/truman/Documents/DataAPI/auto-mpg.csv"
dataframe = pd.read_csv(file_url, engine="python")

## Some quick cleanup of the dataframe:

First making origin more readible, 

Then by converting horsepower into a numeric variable (it is automatically imported as a string type since ? is included in data). replacing null values formerly represented by ? in csv to median

In [3]:
dataframe["origin"].replace({1: "American", 2: "European", 3: "Asian"}, inplace=True)
dataframe["horsepower"] = pd.to_numeric(dataframe["horsepower"], errors="coerce") #sets "?" to NaN
dataframe["horsepower"].fillna(dataframe["horsepower"].median(), inplace=True)

In [4]:
dataframe.shape

(398, 9)

In [5]:
dataframe.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,modelyear,origin,carname
0,18.0,8,307.0,130.0,3504,12.0,70,American,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,American,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,American,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,American,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,American,ford torino


In [6]:
val_dataframe = dataframe.sample(frac=0.2, random_state=1337)
train_dataframe = dataframe.drop(val_dataframe.index)

print(
    "Using %d samples for training and %d for validation"
    % (len(train_dataframe), len(val_dataframe))
)

Using 318 samples for training and 80 for validation


In [7]:
def dataframe_to_dataset(dataframe):
    dataframe = dataframe.copy()
    labels = dataframe.pop("mpg")
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    ds = ds.shuffle(buffer_size=len(dataframe))
    return ds


train_ds = dataframe_to_dataset(train_dataframe)
val_ds = dataframe_to_dataset(val_dataframe)

2022-01-08 00:21:14.740284: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-01-08 00:21:14.740366: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-01-08 00:21:14.740408: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (KernelBlotto): /proc/driver/nvidia/version does not exist
2022-01-08 00:21:14.740847: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
for x, y in train_ds.take(1):
    print("Input:", x)
    print("Target:", y)

Input: {'cylinders': <tf.Tensor: shape=(), dtype=int64, numpy=4>, 'displacement': <tf.Tensor: shape=(), dtype=float64, numpy=119.0>, 'horsepower': <tf.Tensor: shape=(), dtype=float64, numpy=97.0>, 'weight': <tf.Tensor: shape=(), dtype=int64, numpy=2545>, 'acceleration': <tf.Tensor: shape=(), dtype=float64, numpy=17.0>, 'modelyear': <tf.Tensor: shape=(), dtype=int64, numpy=75>, 'origin': <tf.Tensor: shape=(), dtype=string, numpy=b'Asian'>, 'carname': <tf.Tensor: shape=(), dtype=string, numpy=b'datsun 710'>}
Target: tf.Tensor(24.0, shape=(), dtype=float64)


In [9]:
train_ds = train_ds.batch(8)
val_ds = val_ds.batch(8)

In [10]:
from tensorflow.keras.layers import IntegerLookup
from tensorflow.keras.layers import Normalization
from tensorflow.keras.layers import StringLookup


def encode_numerical_feature(feature, name, dataset):
    # Create a Normalization layer for our feature
    normalizer = Normalization()

    # Prepare a Dataset that only yields our feature
    feature_ds = dataset.map(lambda x, y: x[name])
    feature_ds = feature_ds.map(lambda x: tf.expand_dims(x, -1))

    # Learn the statistics of the data
    normalizer.adapt(feature_ds)

    # Normalize the input feature
    encoded_feature = normalizer(feature)
    return encoded_feature


def encode_categorical_feature(feature, name, dataset, is_string):
    lookup_class = StringLookup if is_string else IntegerLookup
    # Create a lookup layer which will turn strings into integer indices
    lookup = lookup_class(output_mode="binary")

    # Prepare a Dataset that only yields our feature
    feature_ds = dataset.map(lambda x, y: x[name])
    feature_ds = feature_ds.map(lambda x: tf.expand_dims(x, -1))

    # Learn the set of possible string values and assign them a fixed integer index
    lookup.adapt(feature_ds)

    # Turn the string input into integer indices
    encoded_feature = lookup(feature)
    return encoded_feature

In [12]:
origin = keras.Input(shape=(1,), name="origin", dtype="string")


# Numerical features
cylinders = keras.Input(shape=(1,), name="cylinders")
displacement = keras.Input(shape=(1,), name="displacement")
horsepower = keras.Input(shape=(1,), name="horsepower")
weight = keras.Input(shape=(1,), name="weight")
modelyear = keras.Input(shape=(1,), name="modelyear")
acceleration = keras.Input(shape=(1,), name="acceleration")

all_inputs = [
    cylinders,
    displacement,
    horsepower,
    weight,
    modelyear,
    acceleration,
    origin,
]

# Integer categorical features
#origin_encoded = encode_categorical_feature(origin, "origin", train_ds, False)
origin_encoded = encode_categorical_feature(origin, "origin", train_ds, True)


# Numerical features
cylinders_encoded = encode_numerical_feature(cylinders, "cylinders", train_ds)
displacement_encoded = encode_numerical_feature(displacement, "displacement", train_ds)
horsepower_encoded = encode_numerical_feature(horsepower, "horsepower", train_ds)
weight_encoded = encode_numerical_feature(weight, "weight", train_ds)
modelyear_encoded = encode_numerical_feature(modelyear, "modelyear", train_ds)
acceleration_encoded = encode_numerical_feature(acceleration, "acceleration", train_ds)

all_features = layers.concatenate(
    [
        cylinders_encoded,
        displacement_encoded,
        horsepower_encoded,
        weight_encoded,
        modelyear_encoded,
        acceleration_encoded,
        origin_encoded,
    ]
)
x = layers.Dense(32, activation="relu")(all_features)
x = layers.Dropout(0.15)(x)
output = layers.Dense(1, activation="relu")(x)
model = keras.Model(all_inputs, output)
model.compile("adam", "mean_squared_error", metrics=["mean_squared_error"])


In [13]:
model.fit(train_ds, epochs=50, validation_data=val_ds)

Epoch 1/50


/home/truman/dataenv/lib/python3.7/site-packages/keras/engine/functional.py:559: UserWarning: Input dict contained keys ['carname'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


40/40 [==============================] - 1s 7ms/step - loss: 607.2640 - mean_squared_error: 607.2640 - val_loss: 537.3174 - val_mean_squared_error: 537.3174
Epoch 2/50
40/40 [==============================] - 0s 3ms/step - loss: 571.8925 - mean_squared_error: 571.8925 - val_loss: 497.6687 - val_mean_squared_error: 497.6687
Epoch 3/50
40/40 [==============================] - 0s 3ms/step - loss: 522.3434 - mean_squared_error: 522.3434 - val_loss: 445.9981 - val_mean_squared_error: 445.9981
Epoch 4/50
40/40 [==============================] - 0s 3ms/step - loss: 459.5651 - mean_squared_error: 459.5651 - val_loss: 386.2224 - val_mean_squared_error: 386.2224
Epoch 5/50
40/40 [==============================] - 0s 3ms/step - loss: 391.7954 - mean_squared_error: 391.7954 - val_loss: 321.6194 - val_mean_squared_error: 321.6194
Epoch 6/50
40/40 [==============================] - 0s 2ms/step - loss: 321.1454 - mean_squared_error: 321.1454 - val_loss: 256.7972 - val_mean_squared_error: 256.7972
Epo

In [34]:
sample = {
    "cylinders":8,
    "displacement":300,
    "horsepower":150,
    "weight":4500,
    "acceleration":10,
    "modelyear":96,
    "origin":"American",    
}


input_dict = {name: tf.convert_to_tensor([value]) for name, value in sample.items()}
predictions = model.predict(input_dict)

print(f"This car has gets an estimated {predictions[0][0]} mpg")


This car has gets an estimated 28.225862503051758 mpg


In [15]:
model.save('/home/truman/Documents/DataAPI/mpg_model_saved')

2022-01-08 00:21:50.299253: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /home/truman/Documents/DataAPI/mpg_model_saved/assets
